In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyvallocation.views import FlexibleViewsProcessor, BlackLittermanProcessor
from pyvallocation.portfolioapi import AssetsDistribution, PortfolioWrapper
from pyvallocation import probabilities, moments

# load daily close data for some ETFs
df = pd.read_csv('../examples/ETF_prices.csv',index_col=0,parse_dates=True)
print('Loaded ', df.columns)

# resample to weekly frequency
weekly_prices = df.resample('W').ffill()

# compute compounded returns (assuming the weekly compounded returns are suitable risk drivers and they are equal to simple returns, we'll relax these assumptions later)
weekly_returns = np.log(weekly_prices).diff().dropna()

# store the returns shape
T, N = weekly_returns.shape

# inputs/parameters
ANNUALIZATION_FACTOR = 52
TARGET_RETURN = 0.06/ANNUALIZATION_FACTOR

Loaded  Index(['DBC', 'GLD', 'SPY', 'TLT'], dtype='object')


In [3]:
# generate uniform probabilities
p_uniform = probabilities.generate_uniform_probabilities(T)
print('Effective number of scenarios of probabilities with uniform dist:', probabilities.compute_effective_number_scenarios(p_uniform))

# estimate mean and cov using the uniform prob.
mu_uniform, cov_uniform = moments.estimate_sample_moments(weekly_returns,p_uniform)

# apply shrinkage to mean and cov
mu_uniform_jorion = moments.shrink_mean_jorion(mu_uniform,cov_uniform,T)
cov_uniform_lw_cc = moments.shrink_covariance_ledoit_wolf(weekly_returns,cov_uniform,target='constant_correlation')

Effective number of scenarios of probabilities with uniform dist: 1006.0000000000003


In [5]:
# Suppose we think S&P 500 will deliver 2% and GLD 5% annualized return
mean_views_1 = {"SPY": 0.02/ANNUALIZATION_FACTOR, 'GLD':0.05/ANNUALIZATION_FACTOR}

# black litterman
bl = BlackLittermanProcessor(
    prior_mean=mu_uniform_jorion,
    prior_cov=cov_uniform_lw_cc,
    mean_views=mean_views_1,
    verbose=True
)
mu_bl, cov_bl = bl.get_posterior()

# flexible views (in this case since we're using mean and cov as input, the class will generate simulated scenarios draw from normal distrubiton)
fv = FlexibleViewsProcessor(
    prior_mean=mu_uniform_jorion,
    prior_cov=cov_uniform_lw_cc,
    mean_views=mean_views_1,
)
mu_fv, cov_fv = fv.get_posterior()

[BL] π source: prior_mean.
[BL] Built P (2, 4), Q (2, 1).
[BL] Ω = τ·diag(P Σ Pᵀ).
[BL] Posterior mean and covariance computed.


In [8]:
w_bl = PortfolioWrapper(AssetsDistribution(mu_bl,cov_bl)).get_portfolios_return_constraint(TARGET_RETURN)
print('\nBL weights:', w_bl, '\n---\n')
w_fl = PortfolioWrapper(AssetsDistribution(mu_fv,cov_fv)).get_portfolios_return_constraint(TARGET_RETURN)
print('FV weights:', w_fl, '\n---\n')

Setting default optimizer, call initialize_optimizer to customize it
No custom constraints specified, using default ones
Setting default optimizer, call initialize_optimizer to customize it
No custom constraints specified, using default ones



BL weights: [DBC    1.211240e-09
GLD    8.002724e-01
SPY    1.997267e-01
TLT    9.638037e-07
dtype: float64] 
---

FV weights: [DBC    3.185827e-08
GLD    9.999997e-01
SPY    4.886972e-08
TLT    2.068196e-07
dtype: float64] 
---

